# FRESH: Framework for Efficient Secure Circuit Synthesis from High-Level Languages

## Setup

The following cell will set up an environment for experimenting with our FRESH toolchain. On Colab, the setup takes about 5 minutes.

The code will download binaries for XLS, Yosys, and the Zig compiler toolchain as well as source code from our work. FRESH is compiled by Zig from source on-the-fly. While Yosys and Zig are fetched from their upstream binary distributions, we modified the XLS codebase and thus provide our own binaries.

In [ ]:
# @title
import os
import pathlib

yosys_version = '0.38_93_g84116c9a3'

# download repository
!curl --show-error -L -o repo.zip https://github.com/encryptogroup/FRESH/archive/refs/heads/main.zip
!unzip -q -o repo.zip

# set up XLS
!curl --show-error -L -o xls-bin.tar.xz.1 'https://anonymous.4open.science/api/repo/paper-review-binaries-081F/file/xls-bin.tar.xz.1?v=7d12e99e&download=true'
!curl --show-error -L -o xls-bin.tar.xz.2 'https://anonymous.4open.science/api/repo/paper-review-binaries-081F/file/xls-bin.tar.xz.2?v=77ede0b4&download=true'
!curl --show-error -L -o xls-bin.tar.xz.3 'https://anonymous.4open.science/api/repo/paper-review-binaries-081F/file/xls-bin.tar.xz.3?v=9361eaf8&download=true'
!cat xls-bin.tar.xz.1 xls-bin.tar.xz.2 xls-bin.tar.xz.3 | tar xJ
xls_bin_path = pathlib.Path('bazel-bin')
xlscc_path = xls_bin_path / 'xls' / 'contrib' / 'xlscc' / 'xlscc'
xls_opt_path = xls_bin_path / 'xls' / 'tools' / 'opt_main'
xls_codegen_path = xls_bin_path / 'xls' / 'tools' / 'codegen_main'
XLSCC = str(xlscc_path.resolve())
XLS_OPT = str(xls_opt_path.resolve())
XLS_CODEGEN = str(xls_codegen_path.resolve())
%env XLSCC={XLSCC}
%env XLS_OPT={XLS_OPT}
%env XLS_CODEGEN={XLS_CODEGEN}

# set up Yosys
!curl -L -O https://repo.anaconda.com/miniconda/Miniconda3-py310_24.1.2-0-Linux-x86_64.sh
!bash Miniconda3-py310_24.1.2-0-Linux-x86_64.sh -b -p conda-env/
conda_prefix_path = pathlib.Path('conda-env')
CONDA_PREFIX = str(conda_prefix_path.resolve())
%env CONDA_PREFIX={CONDA_PREFIX}
!conda-env/bin/conda install -yq -c "litex-hub" yosys={yosys_version}
conda_bin_path = conda_prefix_path / 'bin'
os.environ['PATH'] += os.pathsep + str(conda_bin_path.resolve())

# set up FLUENT
!curl --show-error -L https://ziglang.org/download/0.10.0/zig-linux-x86_64-0.10.0.tar.xz | tar xJ
!zig-linux-x86_64-0.10.0/zig build
fresh_bin_path = pathlib.Path('zig-out') / 'bin'
os.environ['PATH'] += os.pathsep + str(fresh_bin_path.resolve())

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35.8M    0 35.8M    0     0  1198k      0 --:--:--  0:00:30 --:--:--  614k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75.7M    0 75.7M    0     0  7591k      0 --:--:--  0:00:10 --:--:-- 9122k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75.7M    0 75.7M    0     0  9727k      0 --:--:--  0:00:07 --:--:-- 10.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75.7M    0 75.7M    0     0  12.3M      0 --:--:--  0:00:06 --:--:-- 16.2M
env: XLSCC=/content/bazel-bin/xls/contrib/xlscc/xlsc

## Introductory examples

First, we will have a quick look at some simple examples.

### Verilog

The following Verilog code implements a simple 32-bit integer adder.

In [ ]:
%%writefile example_verilog.v

module adder (input wire [31:0] a, input wire [31:0] b, output wire [31:0] c);
  assign c = a + b;
endmodule

Writing paper_example_verilog.v


In [ ]:
!syncirc example_verilog.v -t adder
%pfile example_verilog.syncirc

compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_verilog.spfe` not found.


Let's have a look at the circuit depth and size.

In [ ]:
!stats example_verilog.syncirc

### C/C++

This is the same example implemented in C, made possible with High-Level Synthesis (HLS).

In [ ]:
%%writefile example_c.c

int adder(int a, int b) {
  return a + b;
}

Writing paper_example_c.c


In [ ]:
!syncirc example_c.c -t adder
%pfile example_c.syncirc

running high-level synthesis with XLS... compiling public components... compiling public components... compiling private components [1/1] compiling private components [2/1] merging circuits... Object `paper_example_c.spfe` not found.


In [ ]:
!stats example_c.syncirc